In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn import preprocessing  #preprocessing for label encoding...
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
#Read the train data into input_df
in_df = pd.read_csv("/kaggle/input/titanic/train.csv")
in_df.iloc[759]

PassengerId                                                  760
Survived                                                       1
Pclass                                                         1
Name           Rothes, the Countess. of (Lucy Noel Martha Dye...
Sex                                                       female
Age                                                           33
SibSp                                                          0
Parch                                                          0
Ticket                                                    110152
Fare                                                        86.5
Cabin                                                        B77
Embarked                                                       S
Name: 759, dtype: object

### Preprocessing Data

In [3]:
#Lets get the title from the name
in_df["Title"] = in_df['Name'].map(lambda x: x.split(",")[1].split(" ")[1][:-1])
in_df.loc[in_df['Title']== 'th','Title'] = "Countess"
title = np.unique(in_df['Title'].values)
title

array(['Capt', 'Col', 'Countess', 'Don', 'Dr', 'Jonkheer', 'Lady',
       'Major', 'Master', 'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev',
       'Sir'], dtype=object)

In [4]:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir']:
        return 'Mr'
    elif title in ['Countess', 'Mme','Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [5]:
in_df['Title']=in_df.apply(replace_titles, axis=1)
np.unique(in_df['Title'].values)

array(['Master', 'Miss', 'Mr', 'Mrs'], dtype=object)

In [6]:
#Feature of Name is not important. 
if 'Name' in in_df.columns:
    in_df.drop(columns = "Name",inplace=True)
in_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [7]:
# Let us generate the deck id from the cabin number and fill NaN values as well
cabin_list = in_df['Cabin'].dropna()
cabin_list = cabin_list.tolist()
cabin_list = [i.split(" ") for i in cabin_list]
deck = []
for i in cabin_list:
    for j in i:
        deck.append(j[0])
deck.append('U') # for the NaN labels
deck = np.unique(deck).tolist()
deck

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U']

In [8]:
#Turning cabin number into Deck
#Fillna with Unknown label
in_df['Cabin'].fillna("Unknown",inplace=True)
#For each row find the class and fill in appropriate deck label
in_df['Deck'] = in_df['Cabin'].map(lambda x: x[0])
#Can drop Cabin as number data is not important and the deck info is captured
if 'Cabin' in in_df.columns:
    in_df.drop(columns='Cabin',inplace=True)

In [9]:
#Keep just the ticket number
in_df['Ticket'] = in_df['Ticket'].map(lambda x: x.split(" ")[-1])
in_df.tail()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,Deck
886,887,0,2,male,27.0,0,0,211536,13.00,S,Mr,U
887,888,1,1,female,19.0,0,0,112053,30.00,S,Miss,B
888,889,0,3,female,NaN,1,2,6607,23.45,S,Miss,U
889,890,1,1,male,26.0,0,0,111369,30.00,C,Mr,C
890,891,0,3,male,32.0,0,0,370376,7.75,Q,Mr,U


In [10]:
#Fill the two nan values in embarked col with U for unknown. should think of a better substitute later
in_df["Embarked"].fillna('U',inplace=True)

In [11]:
#Have to fill the null values in the age column as well

#First lets find the mean of the corresponding title category
miss_mean = round(in_df.loc[in_df['Title'] == 'Miss','Age'].dropna().mean(),1)
master_mean = round(in_df.loc[in_df['Title'] == 'Master','Age'].dropna().mean(),1)
mr_mean = round(in_df.loc[in_df['Title'] == 'Mr','Age'].dropna().mean(),1)
mrs_mean = round(in_df.loc[in_df['Title'] == 'Mrs','Age'].dropna().mean(),1)
miss_mean,master_mean,mr_mean,mrs_mean

#Fill the missing ages with the mean of the ages from the respective title category
in_df.loc[in_df['Title'] == 'Miss','Age'] = in_df.loc[in_df['Title'] == 'Miss']['Age'].fillna(miss_mean)
in_df.loc[in_df['Title'] == 'Master','Age'] = in_df.loc[in_df['Title'] == 'Master']['Age'].fillna(master_mean)
in_df.loc[in_df['Title'] == 'Mr','Age'] = in_df.loc[in_df['Title'] == 'Mr']['Age'].fillna(mr_mean)
in_df.loc[in_df['Title'] == 'Mrs','Age'] = in_df.loc[in_df['Title'] == 'Mrs']['Age'].fillna(mrs_mean)
in_df['Age'].values

array([22.  , 38.  , 26.  , 35.  , 35.  , 32.9 , 54.  ,  2.  , 27.  ,
       14.  ,  4.  , 58.  , 20.  , 39.  , 14.  , 55.  ,  2.  , 32.9 ,
       31.  , 36.2 , 35.  , 34.  , 15.  , 28.  ,  8.  , 38.  , 32.9 ,
       19.  , 21.8 , 32.9 , 40.  , 36.2 , 21.8 , 66.  , 28.  , 42.  ,
       32.9 , 21.  , 18.  , 14.  , 40.  , 27.  , 32.9 ,  3.  , 19.  ,
       32.9 , 32.9 , 21.8 , 32.9 , 18.  ,  7.  , 21.  , 49.  , 29.  ,
       65.  , 32.9 , 21.  , 28.5 ,  5.  , 11.  , 22.  , 38.  , 45.  ,
        4.  , 32.9 ,  4.6 , 29.  , 19.  , 17.  , 26.  , 32.  , 16.  ,
       21.  , 26.  , 32.  , 25.  , 32.9 , 32.9 ,  0.83, 30.  , 22.  ,
       29.  , 21.8 , 28.  , 17.  , 33.  , 16.  , 32.9 , 23.  , 24.  ,
       29.  , 20.  , 46.  , 26.  , 59.  , 32.9 , 71.  , 23.  , 34.  ,
       34.  , 28.  , 32.9 , 21.  , 33.  , 37.  , 28.  , 21.  , 32.9 ,
       38.  , 21.8 , 47.  , 14.5 , 22.  , 20.  , 17.  , 21.  , 70.5 ,
       29.  , 24.  ,  2.  , 21.  , 32.9 , 32.5 , 32.5 , 54.  , 12.  ,
       32.9 , 24.  ,

In [12]:
#Let us label encode sex,embarked and deck
le_sex = preprocessing.LabelEncoder()
le_embarked = preprocessing.LabelEncoder()
le_deck = preprocessing.LabelEncoder()
le_title = preprocessing.LabelEncoder()
#Fit the label encoders
le_sex.fit(in_df['Sex'].values)
le_embarked.fit(in_df['Embarked'].values)
le_deck.fit(in_df['Deck'].values)
le_title.fit(in_df['Title'].values)

#Apply lambda function to each of the column to transform
in_df['Sex'] = in_df['Sex'].map(lambda x: le_sex.transform([x])[0])
in_df['Deck'] = in_df['Deck'].map(lambda x: le_deck.transform([x])[0])
in_df['Embarked'] = in_df['Embarked'].map(lambda x: le_embarked.transform([x])[0])
in_df['Title'] = in_df['Title'].map(lambda x: le_title.transform([x])[0])

in_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,Deck
0,1,0,3,1,22.0,1,0,21171,7.2500,2,2,8
1,2,1,1,0,38.0,1,0,17599,71.2833,0,3,2
2,3,1,3,0,26.0,0,0,3101282,7.9250,2,1,8
3,4,1,1,0,35.0,1,0,113803,53.1000,2,3,2
4,5,0,3,1,35.0,0,0,373450,8.0500,2,2,8


In [13]:
in_df.loc[in_df['Ticket'] == "LINE",'Ticket'] = round(in_df.loc[in_df['Ticket'] != 'LINE','Ticket'].dropna().mean(),1)

In [14]:
#Incase we are still left with any nan values
in_df  = in_df.fillna(method='ffill')

#Also round the fare
in_df['Fare'] = in_df['Fare'].map(lambda x: round(x,2))
#Convert ticket from string to float64
in_df['Ticket'] = in_df['Ticket'].map(lambda x: round(float(x),2))

#### Normalize Data

In [15]:
max_ticket = max(in_df.loc[:178,'Ticket'].max(),in_df.loc[180:270,'Ticket'].max(),in_df.loc[272:301,'Ticket'].max(),in_df.loc[303:596,'Ticket'].max())
in_df.loc[179,'Ticket'] = max_ticket
in_df.loc[271,'Ticket'] = max_ticket
in_df.loc[302,'Ticket'] = max_ticket
in_df.loc[597,'Ticket'] = max_ticket

In [16]:
np.where(in_df.values >= np.finfo(np.float64).max)

(array([], dtype=int64), array([], dtype=int64))

In [17]:
in_df['Ticket'].describe()

count    8.910000e+02
mean     3.109120e+05
std      6.823970e+05
min      3.000000e+00
25%      1.487900e+04
50%      1.130280e+05
75%      3.470820e+05
max      3.101317e+06
Name: Ticket, dtype: float64

In [18]:
#Let us normalize mostly all columns
x = in_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
in_df = pd.DataFrame(x_scaled)
in_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.0,1.0,1.0,0.271174,0.125,0.0,0.006825,0.014151,0.666667,0.666667,1.00
1,0.001124,1.0,0.0,0.0,0.472229,0.125,0.0,0.005674,0.139129,0.000000,1.000000,0.25
2,0.002247,1.0,1.0,0.0,0.321438,0.000,0.0,0.999989,0.015459,0.666667,0.333333,1.00
3,0.003371,1.0,0.0,0.0,0.434531,0.125,0.0,0.036694,0.103644,0.666667,1.000000,0.25
4,0.004494,0.0,1.0,1.0,0.434531,0.000,0.0,0.120416,0.015713,0.666667,0.666667,1.00


In [19]:
Y = in_df[1].values.reshape(891,1)
X = in_df.drop(columns=1)
X = np.array(X)
X

array([[0.        , 1.        , 1.        , ..., 0.66666667, 0.66666667,
        1.        ],
       [0.0011236 , 0.        , 0.        , ..., 0.        , 1.        ,
        0.25      ],
       [0.00224719, 1.        , 0.        , ..., 0.66666667, 0.33333333,
        1.        ],
       ...,
       [0.99775281, 1.        , 0.        , ..., 0.66666667, 0.33333333,
        1.        ],
       [0.9988764 , 0.        , 1.        , ..., 0.        , 0.66666667,
        0.25      ],
       [1.        , 1.        , 1.        , ..., 0.33333333, 0.66666667,
        1.        ]])

In [20]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.13, random_state=42)
#Print the dimensions of the train and test(dev) set to verify.
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(Y_dev.shape))

number of training examples = 775
number of dev examples = 116
X_train shape: (775, 11)
Y_train shape: (775, 1)
X_dev shape: (116, 11)
Y_dev shape: (116, 1)


In [21]:
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.float32)

### Create the NN model

In [22]:
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
from keras.layers.advanced_activations import LeakyReLU


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

Using TensorFlow backend.


In [381]:
def TitanicModel(input_shape):
    
    model = tf.keras.models.Sequential()
#     model.add(tf.keras.Input(shape=(input_shape,)))
    model.add(tf.keras.layers.Flatten(input_shape=(input_shape,)))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(512, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer="glorot_uniform",
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer="glorot_uniform",
    bias_initializer="zeros",))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.BatchNormalization(axis=1))
    model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer="glorot_uniform",
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer="glorot_uniform",kernel_regularizer='l2',
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(16, activation='relu', kernel_initializer="glorot_uniform",
    bias_initializer="zeros",))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [382]:
model = TitanicModel(X_train.shape[1])

In [383]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [386]:
model.fit(X_train,Y_train,batch_size=64,epochs=500)

Train on 775 samples
Epoch 1/500
775/775 [==============================] - 0s 138us/sample - loss: 0.3786 - accuracy: 0.8890
Epoch 2/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3775 - accuracy: 0.8981
Epoch 3/500
775/775 [==============================] - 0s 128us/sample - loss: 0.3670 - accuracy: 0.9019
Epoch 4/500
775/775 [==============================] - 0s 136us/sample - loss: 0.3857 - accuracy: 0.8839
Epoch 5/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3627 - accuracy: 0.8929
Epoch 6/500
775/775 [==============================] - 0s 172us/sample - loss: 0.3483 - accuracy: 0.9097
Epoch 7/500
775/775 [==============================] - 0s 141us/sample - loss: 0.3586 - accuracy: 0.9019
Epoch 8/500
775/775 [==============================] - 0s 132us/sample - loss: 0.3475 - accuracy: 0.8968
Epoch 9/500
775/775 [==============================] - 0s 130us/sample - loss: 0.3406 - accuracy: 0.9071
Epoch 10/500
775/775 [============

775/775 [==============================] - 0s 132us/sample - loss: 0.3650 - accuracy: 0.8929
Epoch 79/500
775/775 [==============================] - 0s 141us/sample - loss: 0.3718 - accuracy: 0.8890
Epoch 80/500
775/775 [==============================] - 0s 133us/sample - loss: 0.3974 - accuracy: 0.8748
Epoch 81/500
775/775 [==============================] - 0s 141us/sample - loss: 0.3652 - accuracy: 0.9058
Epoch 82/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3891 - accuracy: 0.8877
Epoch 83/500
775/775 [==============================] - 0s 126us/sample - loss: 0.3812 - accuracy: 0.8981
Epoch 84/500
775/775 [==============================] - 0s 134us/sample - loss: 0.3776 - accuracy: 0.8877
Epoch 85/500
775/775 [==============================] - 0s 130us/sample - loss: 0.3531 - accuracy: 0.9006
Epoch 86/500
775/775 [==============================] - 0s 125us/sample - loss: 0.3459 - accuracy: 0.9084
Epoch 87/500
775/775 [==============================] - 0s 

775/775 [==============================] - 0s 122us/sample - loss: 0.3558 - accuracy: 0.8968
Epoch 155/500
775/775 [==============================] - 0s 129us/sample - loss: 0.3666 - accuracy: 0.8942
Epoch 156/500
775/775 [==============================] - 0s 135us/sample - loss: 0.3639 - accuracy: 0.8903
Epoch 157/500
775/775 [==============================] - 0s 125us/sample - loss: 0.3339 - accuracy: 0.8981
Epoch 158/500
775/775 [==============================] - 0s 128us/sample - loss: 0.3469 - accuracy: 0.9019
Epoch 159/500
775/775 [==============================] - 0s 126us/sample - loss: 0.3821 - accuracy: 0.8865
Epoch 160/500
775/775 [==============================] - 0s 135us/sample - loss: 0.3779 - accuracy: 0.8994
Epoch 161/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3781 - accuracy: 0.8890
Epoch 162/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3685 - accuracy: 0.8877
Epoch 163/500
775/775 [============================

775/775 [==============================] - 0s 139us/sample - loss: 0.3227 - accuracy: 0.9071
Epoch 231/500
775/775 [==============================] - 0s 140us/sample - loss: 0.3341 - accuracy: 0.9058
Epoch 232/500
775/775 [==============================] - 0s 135us/sample - loss: 0.3253 - accuracy: 0.9058
Epoch 233/500
775/775 [==============================] - 0s 177us/sample - loss: 0.3128 - accuracy: 0.9058
Epoch 234/500
775/775 [==============================] - 0s 152us/sample - loss: 0.3126 - accuracy: 0.9097
Epoch 235/500
775/775 [==============================] - 0s 136us/sample - loss: 0.3123 - accuracy: 0.9045
Epoch 236/500
775/775 [==============================] - 0s 134us/sample - loss: 0.3214 - accuracy: 0.9058
Epoch 237/500
775/775 [==============================] - 0s 134us/sample - loss: 0.3260 - accuracy: 0.9032
Epoch 238/500
775/775 [==============================] - 0s 132us/sample - loss: 0.3123 - accuracy: 0.9032
Epoch 239/500
775/775 [============================

775/775 [==============================] - 0s 135us/sample - loss: 0.2787 - accuracy: 0.9200
Epoch 307/500
775/775 [==============================] - 0s 131us/sample - loss: 0.2902 - accuracy: 0.9110
Epoch 308/500
775/775 [==============================] - 0s 138us/sample - loss: 0.2968 - accuracy: 0.9174
Epoch 309/500
775/775 [==============================] - 0s 149us/sample - loss: 0.3004 - accuracy: 0.9135
Epoch 310/500
775/775 [==============================] - 0s 131us/sample - loss: 0.2940 - accuracy: 0.9226
Epoch 311/500
775/775 [==============================] - 0s 137us/sample - loss: 0.3386 - accuracy: 0.9110
Epoch 312/500
775/775 [==============================] - 0s 154us/sample - loss: 0.3112 - accuracy: 0.9097
Epoch 313/500
775/775 [==============================] - 0s 130us/sample - loss: 0.3310 - accuracy: 0.9071
Epoch 314/500
775/775 [==============================] - 0s 138us/sample - loss: 0.3237 - accuracy: 0.9071
Epoch 315/500
775/775 [============================

775/775 [==============================] - 0s 131us/sample - loss: 0.3114 - accuracy: 0.9097
Epoch 383/500
775/775 [==============================] - 0s 131us/sample - loss: 0.3257 - accuracy: 0.9006
Epoch 384/500
775/775 [==============================] - 0s 135us/sample - loss: 0.3197 - accuracy: 0.9071
Epoch 385/500
775/775 [==============================] - 0s 133us/sample - loss: 0.3117 - accuracy: 0.9110
Epoch 386/500
775/775 [==============================] - 0s 130us/sample - loss: 0.3190 - accuracy: 0.9110
Epoch 387/500
775/775 [==============================] - 0s 134us/sample - loss: 0.3561 - accuracy: 0.8929
Epoch 388/500
775/775 [==============================] - 0s 137us/sample - loss: 0.3697 - accuracy: 0.8800
Epoch 389/500
775/775 [==============================] - 0s 130us/sample - loss: 0.3673 - accuracy: 0.8787
Epoch 390/500
775/775 [==============================] - 0s 163us/sample - loss: 0.3628 - accuracy: 0.8877
Epoch 391/500
775/775 [============================

775/775 [==============================] - 0s 128us/sample - loss: 0.3551 - accuracy: 0.8916
Epoch 459/500
775/775 [==============================] - 0s 148us/sample - loss: 0.3487 - accuracy: 0.8916
Epoch 460/500
775/775 [==============================] - 0s 122us/sample - loss: 0.3429 - accuracy: 0.9058
Epoch 461/500
775/775 [==============================] - 0s 126us/sample - loss: 0.3221 - accuracy: 0.9110
Epoch 462/500
775/775 [==============================] - 0s 128us/sample - loss: 0.3054 - accuracy: 0.9071
Epoch 463/500
775/775 [==============================] - 0s 133us/sample - loss: 0.3102 - accuracy: 0.9045
Epoch 464/500
775/775 [==============================] - 0s 128us/sample - loss: 0.3092 - accuracy: 0.9045
Epoch 465/500
775/775 [==============================] - 0s 126us/sample - loss: 0.3220 - accuracy: 0.8942
Epoch 466/500
775/775 [==============================] - 0s 126us/sample - loss: 0.3253 - accuracy: 0.9006
Epoch 467/500
775/775 [============================

In [387]:
#Compute the accuracy for the dev test
preds = model.evaluate(X_dev,Y_dev,batch_size=16)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

116/116 [==============================] - 0s 171us/sample - loss: 0.6611 - accuracy: 0.8534

Loss = 0.6611199492010577
Test Accuracy = 0.8534483


### Predictions

In [162]:
out_df = pd.read_csv("/kaggle/input/titanic/test.csv")
out_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [163]:
out_df["Title"] = out_df['Name'].map(lambda x: x.split(",")[1].split(" ")[1][:-1])
out_df.loc[out_df['Title'] == 'Dona','Title'] = "Mrs"
title = np.unique(out_df['Title'].values)
title

array(['Col', 'Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Ms', 'Rev'],
      dtype=object)

In [164]:
out_df['Title']=out_df.apply(replace_titles, axis=1)
np.unique(out_df['Title'].values)

array(['Master', 'Miss', 'Mr', 'Mrs'], dtype=object)

In [165]:
#Feature of Name is not important. 
if 'Name' in out_df.columns:
    out_df.drop(columns = "Name",inplace=True)
out_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,Mrs


In [166]:
# Let us generate the deck id from the cabin number and fill NaN values as well
cabin_list = out_df['Cabin'].dropna()
cabin_list = cabin_list.tolist()
cabin_list = [i.split(" ") for i in cabin_list]
deck = []
for i in cabin_list:
    for j in i:
        deck.append(j[0])
deck.append('U') # for the NaN labels
deck = np.unique(deck).tolist()
deck

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'U']

In [167]:
#Turning cabin number into Deck
#Fillna with Unknown label
out_df['Cabin'].fillna("Unknown",inplace=True)
#For each row find the class and fill in appropriate deck label
out_df['Deck'] = out_df['Cabin'].map(lambda x: x[0])
#Can drop Cabin as number data is not important and the deck info is captured
if 'Cabin' in out_df.columns:
    out_df.drop(columns='Cabin',inplace=True)

In [168]:
#Keep just the ticket number
out_df['Ticket'] = out_df['Ticket'].map(lambda x: x.split(" ")[-1])
out_df.tail()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,Deck
413,1305,3,male,NaN,0,0,3236,8.0500,S,Mr,U
414,1306,1,female,39.0,0,0,17758,108.9000,C,Mrs,C
415,1307,3,male,38.5,0,0,3101262,7.2500,S,Mr,U
416,1308,3,male,NaN,0,0,359309,8.0500,S,Mr,U
417,1309,3,male,NaN,1,1,2668,22.3583,C,Master,U


In [169]:
#Fill the two nan values in embarked col with U for unknown. should think of a better substitute later
out_df["Embarked"].fillna('U',inplace=True)

In [170]:
#Have to fill the null values in the age column as well

#First lets find the mean of the corresponding title category
miss_mean = round(out_df.loc[out_df['Title'] == 'Miss','Age'].dropna().mean(),1)
master_mean = round(out_df.loc[out_df['Title'] == 'Master','Age'].dropna().mean(),1)
mr_mean = round(out_df.loc[out_df['Title'] == 'Mr','Age'].dropna().mean(),1)
mrs_mean = round(out_df.loc[out_df['Title'] == 'Mrs','Age'].dropna().mean(),1)
miss_mean,master_mean,mr_mean,mrs_mean

#Fill the missing ages with the mean of the ages from the respective title category
out_df.loc[out_df['Title'] == 'Miss','Age'] = out_df.loc[out_df['Title'] == 'Miss']['Age'].fillna(miss_mean)
out_df.loc[out_df['Title'] == 'Master','Age'] = out_df.loc[out_df['Title'] == 'Master']['Age'].fillna(master_mean)
out_df.loc[out_df['Title'] == 'Mr','Age'] = out_df.loc[out_df['Title'] == 'Mr']['Age'].fillna(mr_mean)
out_df.loc[out_df['Title'] == 'Mrs','Age'] = out_df.loc[out_df['Title'] == 'Mrs']['Age'].fillna(mrs_mean)
out_df['Age'].values

array([34.5 , 47.  , 62.  , 27.  , 22.  , 14.  , 30.  , 26.  , 18.  ,
       21.  , 32.2 , 46.  , 23.  , 63.  , 47.  , 24.  , 35.  , 21.  ,
       27.  , 45.  , 55.  ,  9.  , 39.1 , 21.  , 48.  , 50.  , 22.  ,
       22.5 , 41.  , 32.2 , 50.  , 24.  , 33.  , 39.1 , 30.  , 18.5 ,
       21.8 , 21.  , 25.  , 32.2 , 39.  , 32.2 , 41.  , 30.  , 45.  ,
       25.  , 45.  , 32.2 , 60.  , 36.  , 24.  , 27.  , 20.  , 28.  ,
       32.2 , 10.  , 35.  , 25.  , 32.2 , 36.  , 17.  , 32.  , 18.  ,
       22.  , 13.  , 39.1 , 18.  , 47.  , 31.  , 60.  , 24.  , 21.  ,
       29.  , 28.5 , 35.  , 32.5 , 32.2 , 55.  , 30.  , 24.  ,  6.  ,
       67.  , 49.  , 32.2 , 32.2 , 32.2 , 27.  , 18.  , 21.8 ,  2.  ,
       22.  , 32.2 , 27.  , 32.2 , 25.  , 25.  , 76.  , 29.  , 20.  ,
       33.  , 43.  , 27.  , 32.2 , 26.  , 16.  , 28.  , 21.  , 32.2 ,
       32.2 , 18.5 , 41.  , 21.8 , 36.  , 18.5 , 63.  , 18.  , 32.2 ,
        1.  , 36.  , 29.  , 12.  , 32.2 , 35.  , 28.  , 32.2 , 17.  ,
       22.  , 21.8 ,

In [171]:
#Apply lambda function to each of the column to transform
out_df['Sex'] = out_df['Sex'].map(lambda x: le_sex.transform([x])[0])
out_df['Deck'] = out_df['Deck'].map(lambda x: le_deck.transform([x])[0])
out_df['Embarked'] = out_df['Embarked'].map(lambda x: le_embarked.transform([x])[0])
out_df['Title'] = out_df['Title'].map(lambda x: le_title.transform([x])[0])

out_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,Deck
0,892,3,1,34.5,0,0,330911,7.8292,1,2,8
1,893,3,0,47.0,1,0,363272,7.0000,2,3,8
2,894,2,1,62.0,0,0,240276,9.6875,1,2,8
3,895,3,1,27.0,0,0,315154,8.6625,2,2,8
4,896,3,0,22.0,1,1,3101298,12.2875,2,3,8


In [172]:
out_df.loc[out_df['Ticket'] == "LINE",'Ticket'] = round(out_df.loc[out_df['Ticket'] != 'LINE','Ticket'].dropna().mean(),1)

In [173]:
#Incase we are still left with any nan values
out_df  = out_df.fillna(method='ffill')

#Also round the fare
out_df['Fare'] = out_df['Fare'].map(lambda x: round(x,2))
#Convert ticket from string to float64
out_df['Ticket'] = out_df['Ticket'].map(lambda x: round(float(x),2))

In [174]:
# max_ticket = max(out_df.loc[:178,'Ticket'].max(),out_df.loc[180:270,'Ticket'].max(),out_df.loc[272:301,'Ticket'].max(),out_df.loc[303:596,'Ticket'].max())
np.where(in_df.values >= np.finfo(np.float64).max)

(array([], dtype=int64), array([], dtype=int64))

In [175]:
passengerId = out_df['PassengerId'].values

#Let us normalize mostly all columns
x = out_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
out_df = pd.DataFrame(x_scaled)
out_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,1.0,1.0,0.452723,0.000,0.000000,0.106700,0.015283,0.5,0.666667,1.00
1,0.002398,1.0,0.0,0.617566,0.125,0.000000,0.117134,0.013663,1.0,1.000000,1.00
2,0.004796,0.5,1.0,0.815377,0.000,0.000000,0.077475,0.018914,0.5,0.666667,1.00
3,0.007194,1.0,1.0,0.353818,0.000,0.000000,0.101619,0.016903,1.0,0.666667,1.00
4,0.009592,1.0,0.0,0.287881,0.125,0.111111,0.999995,0.023988,1.0,1.000000,1.00
...,...,...,...,...,...,...,...,...,...,...,...
413,0.990408,1.0,1.0,0.422392,0.000,0.000000,0.001043,0.015713,1.0,0.666667,1.00
414,0.992806,0.0,0.0,0.512066,0.000,0.000000,0.005725,0.212558,0.0,1.000000,0.25
415,0.995204,1.0,1.0,0.505473,0.000,0.000000,0.999983,0.014151,1.0,0.666667,1.00
416,0.997602,1.0,1.0,0.422392,0.000,0.000000,0.115856,0.015713,1.0,0.666667,1.00


In [176]:
# Y = out_df[1].values.reshape(418,1)
X = out_df
X = np.array(X)
X.shape

(418, 11)

In [388]:
pred_out = model.predict(X)
for i in pred_out:
    if(i[0]>0.6):
        i[0] = int(1)
    else:
        i[0] = int(0)

In [389]:
passengerId = passengerId.reshape(418)
pred_out = pred_out.reshape(418)
passengerId.shape

(418,)

In [390]:
data_out = {'PassengerId':passengerId, 'Survived':pred_out} 
sub_df = pd.DataFrame(data_out)
sub_df.index  = sub_df.index+1
sub_df['Survived'] = sub_df['Survived'].astype('int')
sub_df

,PassengerId,Survived
1,892,0
2,893,0
3,894,0
4,895,0
5,896,1
...,...,...
414,1305,0
415,1306,1
416,1307,0
417,1308,0


In [392]:
sub_df.to_csv('/kaggle/working/submission_6_0.6.csv',index=False)